In [ ]:
hate_speech_path = "/content/indonesian-abusive-and-hate-speech-twitter-text/data.csv"
kamus_alay_path = "/content/indonesian-abusive-and-hate-speech-twitter-text/new_kamusalay.csv"
id_stopword_path = "/content/indonesian-stoplist/stopwordbahasa.csv"
indobert_path = '/content/drive/MyDrive/Lecture/NLP/indonlu-master/'

# Beginning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.9 MB/s eta 0:00:00


# Load Data

In [ ]:
!pip install opendatasets
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 994.3 kB/s eta 0:00:00


In [ ]:
import opendatasets as od
import pandas as pd

In [ ]:
account = {"username" : "benngki",
           "key" : "ea15cf552e253de01b40b7802fd62815"}

# hate speech text
od.download("https://www.kaggle.com/datasets/ilhamfp31/indonesian-abusive-and-hate-speech-twitter-text", )
# indonesia stop words
od.download("https://www.kaggle.com/datasets/oswinrh/indonesian-stoplist")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: benngki
Your Kaggle Key: ··········


100%|██████████| 814k/814k [00:00<00:00, 104MB/s]


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

 benngki
Your Kaggle Key: ··········


100%|██████████| 2.27k/2.27k [00:00<00:00, 5.95MB/s]

# Exploring Data

In [ ]:
df = pd.read_csv(hate_speech_path, encoding = "latin")
df = df.rename(columns = {"Tweet" : "sentence"})
target_cols = df.columns[1:]
df.head()

,sentence,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [ ]:
dict_alay = pd.read_csv(kamus_alay_path, encoding = 'latin-1',header=None)
dict_alay = dict_alay.rename(columns={0:"original", 1:"replacement"})
dict_alay.head()

,original,replacement
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [ ]:
id_stopword_dict = pd.read_csv(id_stopword_path, header = None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})
id_stopword_dict.head()

,stopword
0,ada
1,adalah
2,adanya
3,adapun
4,agak


# Data Preprocessing

In [ ]:
df.isna().sum()

sentence         0
HS               0
Abusive          0
HS_Individual    0
HS_Group         0
HS_Religion      0
HS_Race          0
HS_Physical      0
HS_Gender        0
HS_Other         0
HS_Weak          0
HS_Moderate      0
HS_Strong        0
dtype: int64

In [ ]:
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text

def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    return text

alay_dict_map = dict(zip(dict_alay['original'], dict_alay['replacement']))
def normalize_alay(text):
    return " ".join([alay_dict_map[word] if word in alay_dict_map.keys() else word for word in text.split()])

def remove_stopword(text):
    text = ' '.join(['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

def preprocess(text):
    text = lowercase(text) # 1
    text = remove_nonaplhanumeric(text) # 2
    text = remove_unnecessary_char(text) # 2
    text = normalize_alay(text) # 3
    text = stemming(text) # 4
    # text = remove_stopword(text) # 5
    return text

In [ ]:
old_df = df.copy()

df["sentence"] = df["sentence"].apply(preprocess)
df.head()

,sentence,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,di saat semua cowok usaha lacak perhati gue ka...,1,1,1,0,0,0,0,0,1,1,0,0
1,siapa yang telat beri tau kamu edan sarap gue ...,0,1,0,0,0,0,0,0,0,0,0,0
2,41 kadang aku pikir kenapa aku tetap percaya p...,0,0,0,0,0,0,0,0,0,0,0,0
3,aku itu aku dan ku tau mata sipit tapi lihat d...,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum cebong kafir sudah lihat dongok dari awal...,1,1,0,1,1,0,0,0,0,0,1,0


In [ ]:
pd.DataFrame({"Before":old_df['sentence'], "After":df['sentence']}).head()

,Before,After
0,- disaat semua cowok berusaha melacak perhatia...,di saat semua cowok usaha lacak perhati gue ka...
1,RT USER: USER siapa yang telat ngasih tau elu?...,siapa yang telat beri tau kamu edan sarap gue ...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",41 kadang aku pikir kenapa aku tetap percaya p...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,aku itu aku dan ku tau mata sipit tapi lihat d...
4,USER USER Kaum cebong kapir udah keliatan dong...,kaum cebong kafir sudah lihat dongok dari awal...


# Split dataset into train, valid, and test

In [ ]:
from fast_ml.model_development import train_valid_test_split

X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = target_cols,
                                                                            train_size=0.7, valid_size=0.1, test_size=0.2)

In [ ]:
df_train = pd.concat([X_train, y_train], axis=1)
df_train.reset_index(drop=True, inplace=True)

df_valid = pd.concat([X_valid, y_valid], axis=1)
df_valid.reset_index(drop=True, inplace=True)

df_test = pd.concat([X_test, y_test], axis=1)
df_test.reset_index(drop=True, inplace=True)

print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(9218, 13)
(1317, 13)
(2634, 13)


In [ ]:
df_train.head()

,sentence,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,aku begitu juga bukan cuma nayeon tapi 9 membe...,0,0,0,0,0,0,0,0,0,0,0,0
1,itu yang di khawatir dari manuver politik dan ...,0,0,0,0,0,0,0,0,0,0,0,0
2,hampir saban tahun mereka kata bahwa ios merek...,0,0,0,0,0,0,0,0,0,0,0,0
3,22 duduk syariat islam sama rendah dengan sumb...,0,0,0,0,0,0,0,0,0,0,0,0
4,kapan nih partai demokrasi indonesia juang nas...,1,0,0,1,0,1,0,0,0,0,0,1


# Finetuning CASA

In [ ]:
import os, sys
sys.path.append(indobert_path)
os.chdir(indobert_path)
# os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertTokenizer, BertConfig, BertForPreTraining
from nltk.tokenize import TweetTokenizer, word_tokenize

from modules.multi_label_classification import BertForMultiLabelClassification
from utils.forward_fn import forward_sequence_multi_classification
from utils.metrics import absa_metrics_fn
from utils.data_utils import AspectBasedSentimentAnalysisProsaDataset, AspectBasedSentimentAnalysisDataLoader

In [ ]:
#
# TAMBAHAN FUNGSI
#
from torch.utils.data import Dataset, DataLoader

class AspectBasedSentimentAnalysisProsaDataset(Dataset):
    # Static constant variable
    ASPECT_DOMAIN = ["HS", "Abusive", "HS_Individual", "HS_Group", "HS_Religion", "HS_Race", "HS_Physical", "HS_Gender", "HS_Other", "HS_Weak", "HS_Moderate", "HS_Strong"]
    LABEL2INDEX = {'no': 0, 'yes': 1}
    INDEX2LABEL = {0: 'no', 1: 'yes'}
    NUM_LABELS = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
    NUM_ASPECTS = 12

    def __init__(self, dataframe, tokenizer, no_special_token=False, *args, **kwargs):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.no_special_token = no_special_token

    def __getitem__(self, index):
        data = self.data.loc[index,:]
        sentence, labels = data['sentence'], [data[aspect] for aspect in self.ASPECT_DOMAIN]
        subwords = self.tokenizer.encode(sentence, add_special_tokens=not self.no_special_token)
        return np.array(subwords), np.array(labels), data['sentence']

    def __len__(self):
        return len(self.data)


class AspectBasedSentimentAnalysisDataLoader(DataLoader):
    def __init__(self, dataset, max_seq_len=512, *args, **kwargs):
        super(AspectBasedSentimentAnalysisDataLoader, self).__init__(dataset=dataset, *args, **kwargs)
        self.num_aspects = dataset.NUM_ASPECTS
        self.collate_fn = self._collate_fn
        self.max_seq_len = max_seq_len

    def _collate_fn(self, batch):
        batch_size = len(batch)
        max_seq_len = max(map(lambda x: len(x[0]), batch))
        max_seq_len = min(self.max_seq_len, max_seq_len)

        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        label_batch = np.zeros((batch_size, self.num_aspects), dtype=np.int64)

        seq_list = []

        for i, (subwords, label, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            label_batch[i,:] = label

            seq_list.append(raw_seq)

        return subword_batch, mask_batch, label_batch, seq_list

In [ ]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [ ]:
# Set random seed
set_seed(26092020)

# Load Model

In [ ]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = max(AspectBasedSentimentAnalysisProsaDataset.NUM_LABELS)
config.num_labels_list = AspectBasedSentimentAnalysisProsaDataset.NUM_LABELS

# Instantiate model
model = BertForMultiLabelClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Some weights of BertForMultiLabelClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifiers.10.weight', 'classifiers.1.weight', 'classifiers.9.bias', 'classifiers.4.weight', 'classifiers.5.weight', 'classifiers.5.bias', 'classifiers.1.bias', 'classifiers.11.bias', 'classifiers.8.weight', 'classifiers.7.bias', 'classifiers.7.weight', 'classifiers.9.weight', 'classifiers.2.weight', 'classifiers.3.weight', 'classifiers.4.bias', 'classifiers.6.bias', 'classifiers.3.bias', 'classifiers.0.weight', 'classifiers.0.bias', 'classifiers.2.bias', 'classifiers.8.bias', 'classifiers.10.bias', 'classifiers.6.weight', 'classifiers.11.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)
model = model.cuda()

In [ ]:
model

BertForMultiLabelClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

In [ ]:
count_param(model)

124459800

# Prepare Dataset

In [ ]:
train_dataset = AspectBasedSentimentAnalysisProsaDataset(df_train, tokenizer, lowercase=True)
valid_dataset = AspectBasedSentimentAnalysisProsaDataset(df_valid, tokenizer, lowercase=True)
test_dataset = AspectBasedSentimentAnalysisProsaDataset(df_test, tokenizer, lowercase=True)

train_loader = AspectBasedSentimentAnalysisDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=50, num_workers=50, shuffle=True)
valid_loader = AspectBasedSentimentAnalysisDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=50, num_workers=50, shuffle=False)
test_loader = AspectBasedSentimentAnalysisDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=50, num_workers=50, shuffle=False)

In [ ]:
w2i, i2w = AspectBasedSentimentAnalysisProsaDataset.LABEL2INDEX, AspectBasedSentimentAnalysisProsaDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'no': 0, 'yes': 1}
{0: 'no', 1: 'yes'}


# Test model on sample sentences

In [ ]:
# function for predict
def predict(text):
  prep_text = preprocess(text)
  subwords = tokenizer.encode(prep_text)
  subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

  logits = model(subwords)[0]
  labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

  # print(f'Text: {text}')
  # for i, label in enumerate(labels):
  #     print(f'Label `{AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]}` : {i2w[label]} ({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)')

  output = {}
  for i, label in enumerate(labels):
      # print(f'Label `{AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]}` : {i2w[label]} ({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)')
      temp = f'({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)'
      output[AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]] = [i2w[label], temp]
  return output

In [ ]:
predict("tolol ah lu jadi manusia")

{'HS': ['yes', '(53.802%)'],
 'Abusive': ['no', '(60.018%)'],
 'HS_Individual': ['no', '(69.786%)'],
 'HS_Group': ['yes', '(60.288%)'],
 'HS_Religion': ['no', '(66.969%)'],
 'HS_Race': ['no', '(67.751%)'],
 'HS_Physical': ['yes', '(69.724%)'],
 'HS_Gender': ['no', '(50.711%)'],
 'HS_Other': ['no', '(54.504%)'],
 'HS_Weak': ['yes', '(52.371%)'],
 'HS_Moderate': ['yes', '(53.545%)'],
 'HS_Strong': ['yes', '(55.156%)']}

In [ ]:
predict("goblok banget sih lu")

{'HS': ['yes', '(55.804%)'],
 'Abusive': ['no', '(55.814%)'],
 'HS_Individual': ['no', '(71.165%)'],
 'HS_Group': ['yes', '(62.271%)'],
 'HS_Religion': ['no', '(61.884%)'],
 'HS_Race': ['no', '(67.844%)'],
 'HS_Physical': ['yes', '(65.078%)'],
 'HS_Gender': ['no', '(51.530%)'],
 'HS_Other': ['no', '(56.228%)'],
 'HS_Weak': ['yes', '(51.817%)'],
 'HS_Moderate': ['yes', '(50.426%)'],
 'HS_Strong': ['no', '(52.110%)']}

In [ ]:
predict("males gue kenalan sama orang kek elu")

{'HS': ['no', '(52.219%)'],
 'Abusive': ['no', '(59.971%)'],
 'HS_Individual': ['no', '(71.795%)'],
 'HS_Group': ['yes', '(52.679%)'],
 'HS_Religion': ['no', '(66.801%)'],
 'HS_Race': ['no', '(72.858%)'],
 'HS_Physical': ['yes', '(70.414%)'],
 'HS_Gender': ['yes', '(50.409%)'],
 'HS_Other': ['no', '(51.824%)'],
 'HS_Weak': ['yes', '(50.255%)'],
 'HS_Moderate': ['yes', '(51.043%)'],
 'HS_Strong': ['yes', '(56.154%)']}

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
class Detector:
  df = None
  df_predict = None
  df_comb = None

  def __init__(self, data_path, tweet_column_name):
    self.data_path = data_path
    self.tweet_column_name = tweet_column_name
    self._process()
    self._predict()
    self._calcAccuracyModel()
    self._combineDf()

  def _process(self):
    df = pd.read_csv(self.data_path, encoding = "latin")
    df.rename(columns={self.tweet_column_name : 'tweet'}, inplace=True)
    self.df = df

  def _predict(self):
    new_df = pd.DataFrame(self.df['tweet'])

    model_pred = [predict(i) for i in new_df['tweet']]

    for label in AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN:
      for data in model_pred:
        new_df[label] = [w2i[i[label][0]] for i in model_pred]

    self.df_predict = new_df

  def _calcAccuracyModel(self):
    self.accuracy = round(accuracy_score(self.df_predict[['HS', 'Abusive']], self.df[['HS', 'Abusive']]) * 100, 2)

  def _combineDf(self):
    df = pd.DataFrame(self.df[['tweet', "HS", "Abusive"]])
    df.rename(columns={"HS" : "HS_origin", "Abusive" : "Abusive_origin"}, inplace=True)
    df = pd.concat([df, self.df_predict[['HS', "Abusive"]]], axis=1)
    df.rename(columns={"HS" : "HS_pred", "Abusive" : "Abusive_pred"}, inplace=True)
    df.reset_index()
    self.df_comb = df

  def getHateSpecchPercentage(self):
    return round(len(anies.df_predict[anies.df_predict['HS'] == 1])/len(anies.df_predict) * 100, 2)

  def getAbusivePercentage(self):
    return round(len(anies.df_predict[anies.df_predict['Abusive'] == 1])/len(anies.df_predict) * 100, 2)

  def getAccuracy(self):
    return self.accuracy

  # jumlah hatespeech

In [ ]:
df = pd.read_csv(hate_speech_path, encoding = "latin")
df = df.rename(columns = {"Tweet" : "sentence"})

In [ ]:
df.columns

Index(['sentence', 'HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion',
       'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak',
       'HS_Moderate', 'HS_Strong'],
      dtype='object')

In [ ]:
label_test = ['Tweet', 'HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion',
       'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak',
       'HS_Moderate', 'HS_Strong']

# anies
anies = Detector(hate_speech_path, label_test[0])
print("Accuracy model :", anies.getAccuracy())
print("Persentase Hate Speech :", anies.getHateSpecchPercentage())
print("Persentase Abusive :", anies.getAbusivePercentage())

# Fine Tuning & Evaluation

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)
model = model.cuda()

In [ ]:
# Train
n_epochs = 15
# n_epochs = 1
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_multi_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = absa_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)

    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]
        loss, batch_hyp, batch_label = forward_sequence_multi_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = absa_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))

    metrics = absa_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

In [ ]:
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_sequence_multi_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('pred.txt', index=False)

print(df)

# Test fine-tuned model on sample sentences

In [ ]:
predict('tungguin gue napa sih jancuk amat loe')

In [ ]:
predict("tolol dah ditungguin juga bgst'")

In [ ]:
predict("mic gua jgn dimatiin bgst!!!")

# Check test and val score

In [ ]:
class Detector:
  df = None
  df_predict = None
  df_comb = None

  def __init__(self, data_path, tweet_column_name):
    self.data_path = data_path
    self.tweet_column_name = tweet_column_name
    self._process()
    self._predict()
    self._calcAccuracyModel()
    self._combineDf()

  def _process(self):
    df = pd.read_csv(self.data_path, delimiter=',')
    df.rename(columns={self.tweet_column_name : 'tweet'}, inplace=True)
    self.df = df

  def _predict(self):
    new_df = pd.DataFrame(self.df['tweet'])

    model_pred = [predict(i) for i in new_df['tweet']]

    for label in AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN:
      for data in model_pred:
        new_df[label] = [w2i[i[label][0]] for i in model_pred]

    self.df_predict = new_df

  def _calcAccuracyModel(self):
    self.accuracy = round(accuracy_score(self.df_predict[['HS']], self.df[['HS']]) * 100, 2)

  def _combineDf(self):
    # df = pd.DataFrame(self.df[['tweet', "HS", "Abusive"]])
    # df.rename(columns={"HS" : "HS_origin", "Abusive" : "Abusive_origin"}, inplace=True)
    # df = pd.concat([df, self.df_predict[['HS', "Abusive"]]], axis=1)
    # df.rename(columns={"HS" : "HS_pred", "Abusive" : "Abusive_pred"}, inplace=True)
    # df.reset_index()
    # self.df_comb = df
    df = pd.DataFrame(self.df[['tweet', "HS"]])
    df.rename(columns={"HS" : "HS_origin"}, inplace=True)
    df = pd.concat([df, self.df_predict[['HS']]], axis=1)
    df.rename(columns={"HS" : "HS_pred"}, inplace=True)
    df.reset_index()
    self.df_comb = df

  def getHateSpecchPercentage(self):
    return round(len(anies.df_predict[anies.df_predict['HS'] == 1])/len(anies.df_predict) * 100, 2)

  # def getAbusivePercentage(self):
  #   return round(len(anies.df_predict[anies.df_predict['Abusive'] == 1])/len(anies.df_predict) * 100, 2)

  def getAccuracy(self):
    return self.accuracy